## 安裝套件

In [ ]:
!pip install jieba
!pip install emoji langdetect
!pip install datasets
!pip install lingua-language-detector

In [ ]:
import sys
!{sys.executable} -m pip install jieba emoji langdetect pytz torch lingua-language-detector datasets openpyxl

## 引入套件

In [ ]:
pip install transformers

In [ ]:
import pandas as pd
import jieba
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import numpy as np

import re
import emoji
import langdetect
from langdetect import detect
from lingua import LanguageDetectorBuilder, Language, IsoCode639_1
from datetime import datetime
import pytz
from datasets import Dataset
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder
from transformers import AutoTokenizer, AutoModel
import torch

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# 設定好路徑 (後面都是使用相對路徑)
base_path = '/content/drive/My Drive/SMA'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
'''
# 讀取資料（請確認你的 Excel 路徑）
# df = pd.read_excel(base_path+"/threads.xlsx")
df = pd.read_excel("threads.xlsx", engine='openpyxl')
df
'''

'\n# 讀取資料（請確認你的 Excel 路徑）\n# df = pd.read_excel(base_path+"/threads.xlsx")\ndf = pd.read_excel("threads.xlsx", engine=\'openpyxl\')\ndf\n'

In [ ]:
from google.colab import files
import pandas as pd

uploaded = files.upload()

import io
file_name = list(uploaded.keys())[0]
df = pd.read_excel(io.BytesIO(uploaded[file_name]), engine='openpyxl')

df.head()


Saving threads-2.xlsx to threads-2 (1).xlsx


,author,post_time,topic,time_info,content,has_photo,has_video,like_count,reply_count,repost_count,share_count,view_count,followers_count,post_url,scrape_time
0,ayofvr,2025-04-29T10:58:39.000Z,NaN,3小時,Thank you God for another day.,N,N,190,3,23,NaN,"3,197",141073,https://www.threads.net/@ayofvr/post/DJBymf8uTrK,2025-04-29T22:27:40.176749
1,NaN,2025-04-29T12:33:44.000Z,NaN,1小時,百達翡麗？ 沒有下限的網路病態！,Y,N,196,16,NaN,6,3 萬,77683,https://www.threads.net/@ban.mei.onnnnni/post/...,2025-04-29T22:27:54.964788
2,ribboworld2021,2025-04-29T04:39:46.000Z,小一日常,9小時,考完期中考，成績都還沒出來，小一女兒就自信的對我說：「我真羨慕妳生了一個天才！」,N,N,75,6,NaN,NaN,"4,559",99,https://www.threads.net/@ribboworld2021/post/D...,2025-04-29T22:28:09.873641
3,ayofvr,2025-04-29T11:25:22.000Z,NaN,3小時,Just be strong. Confident. Hopeful. Intellectu...,N,N,83,5,3,1,"1,967",141093,https://www.threads.net/@ayofvr/post/DJB1qP5OmzP,2025-04-29T22:28:24.726576
4,jose_ykc,2025-04-29T10:36:19.000Z,輔仁大學,3小時,日文輔系老師上課內容之一AiScReam 歌詞導讀,Y,Y,"4,334",55,513,"2,460",10 萬,65,https://www.threads.net/@jose_ykc/post/DJBvpGI...,2025-04-29T22:28:39.393706


In [ ]:
# === 語言偵測修正版===
lingua_detector = LanguageDetectorBuilder.from_all_languages().with_preloaded_language_models().build()
lingua_available = True
def detect_lang_with_preprocessing_lingua(text):
    original_text = text

    # 若是 NaN 或空字串就回傳 "unknown"
    if pd.isna(text):
        return "unknown"
    text = str(text).strip()
    if not text:
        return "unknown"

    # 移除 URL、@標記、 #hashtag、emoji、多餘空白
    try:
      text_cleaned = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
      text_cleaned = re.sub(r'@\w+', '', text_cleaned)
      text_cleaned = re.sub(r'#\w+', '', text_cleaned)
      text_cleaned = emoji.replace_emoji(text_cleaned, replace='')
      text_cleaned = re.sub(r'\s+', ' ', text_cleaned).strip()
    except Exception as e:
      return "error_state_preprocessing"

    # 若這些清理完後變成空字串
    if not text_cleaned:
      return "empty_after_clean"

    # 若文字中超過 30% 是中文，就直接判定為 "Ch"（中文）
    try:
      chinese_chars = re.findall(r'[\u4e00-\u9fff]', text_cleaned)
      text_len = len(text_cleaned)
      ratio = len(chinese_chars) / max(text_len, 1)
      chinese_threshold = 0.3
      if ratio > chinese_threshold:
        return "Ch"

      # 呼叫 lingua 偵測語言
      detected_language = lingua_detector.detect_language_of(text_cleaned)

      # 若 lingua 判定是中文（'ZH'），則回傳 "Ch"，其餘語言以小寫的 ISO 639-1 回傳（如 en, ja, fr）
      # 若無法偵測出語言，回傳 "unknown"
      if detected_language is not None:
        iso_code = detected_language.iso_code_639_1.name
        if iso_code == 'ZH':
          return "Ch"
        else:
          return iso_code.lower()
      else:
        return "unknown"

    except Exception as e:
      return "unknown"

## 清洗數據V1

In [ ]:
# === 數值欄位清洗（萬字、逗號格式處理）===
def parse_count(value):
    # 將文字數字（如 "1,234"、"2.5萬"）統一轉為整數（int）
    if pd.isna(value): return 0
    value = str(value).replace(",", "")
    # "萬" 的部分會乘上 10,000 做轉換
    # 無法處理的格式就回傳 0
    if "萬" in value:
        return int(float(value.replace("萬", "")) * 10000)
    try:
        return int(float(value))
    except:
        return 0

for col in ["like_count", "view_count", "share_count", "repost_count", "reply_count"]:
    df[col] = df[col].apply(parse_count)

# === 布林欄位處理 ===
# 將原始欄位（Y/N）轉換為 True/False
# 處理過程會去除空白、轉成大寫
df["has_photo"] = df["has_photo"].apply(lambda x: str(x).strip().upper() == "Y")
df["has_video"] = df["has_video"].apply(lambda x: str(x).strip().upper() == "Y")

# === emoji 萃取與統計 ===
# 檢查是否為文字型別，如果是文字，從中萃取出所有 emoji 字元並串接成字串回傳
def extract_emojis(text):
    if not isinstance(text, str): return ""
    return "".join([ch for ch in text if ch in emoji.EMOJI_DATA])

df["emojis"] = df["content"].apply(extract_emojis)
df["emoji_count"] = df["emojis"].apply(len)

# # === 語言偵測修正版===
# def detect_lang_custom(text):
#     try:
#         text = str(text)
#         chinese_chars = re.findall(r'[\u4e00-\u9fff]', text)
#         if len(chinese_chars) / max(len(text), 1) > 0.3:
#             return "Ch"
#         return detect(text)
#     except:
#         return "unknown"

# 使用先前定義好的語言偵測函數 detect_lang_with_preprocessing_lingua()，處理每篇文章的語言判定
df["lang"] = df["content"].apply(detect_lang_with_preprocessing_lingua)

# === scrape_time 處理（轉換時區 + 抽取星期與小時）===
# 將時間欄位轉為台北時區，額外抽出格式化後的時間字串、星期幾、小時(0–23）
df["scrape_time_origin"] = pd.to_datetime(df["scrape_time"], utc=True).dt.tz_convert("Asia/Taipei")
df["scrape_time"]  = df["scrape_time_origin"].dt.strftime("%Y年%m月%d日 %H:%M")
df["post_weekday"] = df["scrape_time_origin"].dt.day_name()
df["post_hour"] = df["scrape_time_origin"].dt.hour

# === 是否為高流量文章（破萬）===
# 超過等於 10,000 瀏覽為 1，其餘為 0
df["viral"] = (df["view_count"] >= 10000).astype(int)

# === 是否使用問號、驚嘆號 ===
df["has_question"] = df["content"].apply(lambda x: "？" in str(x) or "?" in str(x))
df["has_exclaim"] = df["content"].apply(lambda x: "！" in str(x) or "!" in str(x))

# === 儲存結果 ===
df.to_csv("threads_cleaned_v1.csv",encoding='utf_8_sig', index=False)
print("✅ 處理完成，已輸出 threads_cleaned_v1.csv")
display(df)


✅ 處理完成，已輸出 threads_cleaned_v1.csv


,author,post_time,topic,time_info,content,has_photo,has_video,like_count,reply_count,repost_count,...,scrape_time,emojis,emoji_count,lang,scrape_time_origin,post_weekday,post_hour,viral,has_question,has_exclaim
0,ayofvr,2025-04-29T10:58:39.000Z,NaN,3小時,Thank you God for another day.,False,False,190,3,23,...,2025年04月30日 06:27,,0,en,2025-04-30 06:27:40.176749+08:00,Wednesday,6,0,False,False
1,NaN,2025-04-29T12:33:44.000Z,NaN,1小時,百達翡麗？ 沒有下限的網路病態！,True,False,196,16,0,...,2025年04月30日 06:27,,0,Ch,2025-04-30 06:27:54.964788+08:00,Wednesday,6,1,True,True
2,ribboworld2021,2025-04-29T04:39:46.000Z,小一日常,9小時,考完期中考，成績都還沒出來，小一女兒就自信的對我說：「我真羨慕妳生了一個天才！」,False,False,75,6,0,...,2025年04月30日 06:28,,0,Ch,2025-04-30 06:28:09.873641+08:00,Wednesday,6,0,False,True
3,ayofvr,2025-04-29T11:25:22.000Z,NaN,3小時,Just be strong. Confident. Hopeful. Intellectu...,False,False,83,5,3,...,2025年04月30日 06:28,,0,en,2025-04-30 06:28:24.726576+08:00,Wednesday,6,0,False,False
4,jose_ykc,2025-04-29T10:36:19.000Z,輔仁大學,3小時,日文輔系老師上課內容之一AiScReam 歌詞導讀,True,True,4334,55,513,...,2025年04月30日 06:28,,0,Ch,2025-04-30 06:28:39.393706+08:00,Wednesday,6,1,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3096,leighton.williams,2025年04月29日 07:30,NaN,8小時,First tasting in California,False,False,7,0,0,...,2025年04月30日 00:23,,0,en,2025-04-30 00:23:40.328046+08:00,Wednesday,0,0,False,False
3097,cape__man,2025年04月29日 06:31,NaN,9小時,I hoped it would have been better.,False,False,2,0,0,...,2025年04月30日 00:23,,0,en,2025-04-30 00:23:55.063517+08:00,Wednesday,0,0,False,False
3098,simimoonlight,2025年04月29日 06:26,NaN,9小時,I can’t wait to watch Beyoncé on TikTok tonigh...,False,False,2656,29,280,...,2025年04月30日 00:24,🫶🏿🥹,3,en,2025-04-30 00:24:39.870994+08:00,Wednesday,0,1,False,False
3099,other98,2025年04月29日 12:31,NaN,3小時,"Tonight, Canada just proved that they have a h...",False,False,10000,214,214,...,2025年04月30日 00:24,🙌,1,en,2025-04-30 00:24:54.669936+08:00,Wednesday,0,1,False,True


# SnowNLP TextBlob 情緒分析

In [ ]:
pip install snownlp
from snownlp import SnowNLP

#情緒分析（越接近1越正面）
# df["sentiment_score"] = df["content"].apply(lambda x: SnowNLP(str(x)).sentiments if pd.notna(x) else 0.5)

from snownlp import SnowNLP
from textblob import TextBlob
import numpy as np

def get_sentiment_score(text, lang):
    if not isinstance(text, str):
        return 0.5  # 預設中性
    try:
        if lang == "Ch":
            return SnowNLP(text).sentiments
        elif lang == "en":
            return (TextBlob(text).sentiment.polarity + 1) / 2  # 將 -1~1 映射到 0~1
        else:
            return 0.5
    except:
        return 0.5

df["sentiment_score"] = df.apply(lambda row: get_sentiment_score(row["content"], row["lang"]), axis=1)


def classify_sentiment(score):
    if score > 0.66:
        return "positive"
    elif score < 0.33:
        return "negative"
    else:
        return "neutral"
df["sentiment_label"] = df["sentiment_score"].apply(classify_sentiment)

text2emotion 或 transformers 提供的情緒分類模型來擴展為：

喜悅 (joy)
悲傷 (sadness)
生氣 (anger)
驚訝 (surprise)
恐懼 (fear)

In [ ]:
pip install text2emotion
import text2emotion as te
df[['joy', 'anger', 'sadness', 'fear', 'surprise']] = df['content'].apply(lambda x: pd.Series(te.get_emotion(str(x))))


# 人稱視角分析

In [ ]:
#人稱視角分析
def detect_person_view(text):
    if not isinstance(text, str): return "none"
    text = text.lower()

    first_person = any(p in text for p in ["我", "我們", "i", "we"])
    second_person = any(p in text for p in ["你", "你們", "you"])
    third_person = any(p in text for p in ["他", "她", "他們", "她們", "he", "she", "they", "them"])

    person_flags = [first_person, second_person, third_person]
    if person_flags.count(True) > 1:
        return "mixed"
    elif first_person:
        return "1st"
    elif second_person:
        return "2nd"
    elif third_person:
        return "3rd"
    else:
        return "none"

df["person_view"] = df["content"].apply(detect_person_view)
df["has_1st"] = df["person_view"].apply(lambda x: x in ["1st", "mixed"])
df["has_2nd"] = df["person_view"].apply(lambda x: x in ["2nd", "mixed"])
df["has_3rd"] = df["person_view"].apply(lambda x: x in ["3rd", "mixed"])


print(df[["sentiment_score", "person_view"]].describe(include="all"))


        sentiment_score person_view
count       3101.000000        3101
unique              NaN           5
top                 NaN         1st
freq                NaN        1148
mean           0.344515         NaN
std            0.336216         NaN
min            0.000000         NaN
25%            0.030957         NaN
50%            0.253942         NaN
75%            0.582955         NaN
max            1.000000         NaN


## 清洗數據V2

In [ ]:
# --- 文章長度 ---
df["content_length"] = df["content"].apply(lambda x: len(str(x)))

# --- 是否包含網址 ---
df["has_url"] = df["content"].apply(lambda x: "http" in str(x) or "www." in str(x))

# --- 是否包含 @標記他人 ---
df["has_mention"] = df["content"].apply(lambda x: "@" in str(x))

# --- 是否使用 Hashtag ---
df["has_hashtag"] = df["content"].apply(lambda x: "#" in str(x))

# 貼文主題字詞提取（可後續做 TF-IDF 或主題建模）
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(max_features=100, stop_words='english')
word_matrix = vectorizer.fit_transform(df['content'].astype(str))

# 將常見詞語提取出來
keywords = vectorizer.get_feature_names_out()

# 是否為深夜或白天貼文（時間段分類）
def time_period(hour):
    if 5 <= hour < 12:
        return "morning"
    elif 12 <= hour < 17:
        return "afternoon"
    elif 17 <= hour < 22:
        return "evening"
    else:
        return "night"

# 判斷是否合法的time_info格式
def is_valid_time_info(text):
    if pd.isna(text):
        return False
    text = str(text).strip()
    return bool(re.match(r"^\d+\s*(分鐘|小時|天)$", text))

df = df[df["time_info"].apply(is_valid_time_info)].copy()

# 將time_info 統一轉為小時
def convert_time_info(text):
    if pd.isna(text):
        return 0
    text = str(text)
    if "分鐘" in text:
        match = re.search(r"(\d+)", text)
        return int(match.group(1)) / 60
    elif "小時" in text:
        match = re.search(r"(\d+)", text)
        return int(match.group(1))
    elif "天" in text:
        match = re.search(r"(\d+)", text)
        return int(match.group(1)) * 24
    elif "週" in text or "禮拜" in text:
        match = re.search(r"(\d+)", text)
        return int(match.group(1)) * 7 * 24
    else:
        return 0

df["time_elapsed_hours"] = df["time_info"].apply(convert_time_info)
df["post_period"] = df["post_hour"].apply(time_period)

cols_to_show_first = ['author', 'content', 'content_length', 'lang', 'scrape_time', 'post_weekday', 'post_hour', 'post_period', 'viral']
df = df[cols_to_show_first + [col for col in df.columns if col not in cols_to_show_first]]
df.to_csv("threads_cleaned_v2.csv",encoding='utf_8_sig',index=False)
print("✅ 處理完成，已輸出 threads_cleaned_v2.csv")
display(df)

✅ 處理完成，已輸出 threads_cleaned_v2.csv


,author,content,content_length,lang,scrape_time,post_weekday,post_hour,post_period,viral,post_time,...,emoji_count,scrape_time_origin,has_question,has_exclaim,sentiment_score,person_view,has_url,has_mention,has_hashtag,time_elapsed_hours
0,ayofvr,Thank you God for another day.,30,en,2025年04月30日 06:27,Wednesday,6,morning,0,2025-04-29T10:58:39.000Z,...,0,2025-04-30 06:27:40.176749+08:00,False,False,0.802086,mixed,False,False,False,3.0
1,NaN,百達翡麗？ 沒有下限的網路病態！,16,Ch,2025年04月30日 06:27,Wednesday,6,morning,1,2025-04-29T12:33:44.000Z,...,0,2025-04-30 06:27:54.964788+08:00,True,True,0.673785,none,False,False,False,1.0
2,ribboworld2021,考完期中考，成績都還沒出來，小一女兒就自信的對我說：「我真羨慕妳生了一個天才！」,40,Ch,2025年04月30日 06:28,Wednesday,6,morning,0,2025-04-29T04:39:46.000Z,...,0,2025-04-30 06:28:09.873641+08:00,False,True,0.677911,1st,False,False,False,9.0
3,ayofvr,Just be strong. Confident. Hopeful. Intellectu...,69,en,2025年04月30日 06:28,Wednesday,6,morning,0,2025-04-29T11:25:22.000Z,...,0,2025-04-30 06:28:24.726576+08:00,False,False,0.347473,1st,False,False,False,3.0
4,jose_ykc,日文輔系老師上課內容之一AiScReam 歌詞導讀,25,Ch,2025年04月30日 06:28,Wednesday,6,morning,1,2025-04-29T10:36:19.000Z,...,0,2025-04-30 06:28:39.393706+08:00,False,False,0.164487,1st,False,False,False,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3096,leighton.williams,First tasting in California,27,en,2025年04月30日 00:23,Wednesday,0,night,0,2025年04月29日 07:30,...,0,2025-04-30 00:23:40.328046+08:00,False,False,0.447678,1st,False,False,False,8.0
3097,cape__man,I hoped it would have been better.,34,en,2025年04月30日 00:23,Wednesday,0,night,0,2025年04月29日 06:31,...,0,2025-04-30 00:23:55.063517+08:00,False,False,0.149808,1st,False,False,False,9.0
3098,simimoonlight,I can’t wait to watch Beyoncé on TikTok tonigh...,51,en,2025年04月30日 00:24,Wednesday,0,night,1,2025年04月29日 06:26,...,3,2025-04-30 00:24:39.870994+08:00,False,False,0.025393,1st,False,False,False,9.0
3099,other98,"Tonight, Canada just proved that they have a h...",77,en,2025年04月30日 00:24,Wednesday,0,night,1,2025年04月29日 12:31,...,1,2025-04-30 00:24:54.669936+08:00,False,True,0.205699,mixed,False,False,False,3.0


## Self-Attention 模組處理文字資料（測試中 因未寫完可先跳過）

In [ ]:
# BERT 會把 [EMOJI], [CONTENT] 當作分界的詞來理解
def build_semantic_text(row):
    parts = []

    if pd.notna(row["author"]):
        parts.append(f"[AUTHOR] {row['author']}")

    if pd.notna(row["topic"]):
        parts.append(f"[TOPIC] {row['topic']}")

    if pd.notna(row["emojis"]):
        emoji_text = " ".join(row["emojis"])
        parts.append(f"[EMOJI] {emoji_text}")

    if pd.notna(row["content"]):
        parts.append(f"[CONTENT] {row['content']}")

    return " ".join(parts)


# 建立 semantic_text，不影響 df 本身
semantic_text_series = df.apply(build_semantic_text, axis=1)

# 若你想要新 DataFrame：
df_semantic = df.copy()
df_semantic["semantic_text"] = semantic_text_series
df_semantic


,author,content,content_length,lang,scrape_time,post_weekday,post_hour,post_period,viral,post_time,...,scrape_time_origin,has_question,has_exclaim,sentiment_score,person_view,has_url,has_mention,has_hashtag,time_elapsed_hours,semantic_text
0,ayofvr,Thank you God for another day.,30,en,2025年04月30日 06:27,Wednesday,6,morning,0,2025-04-29T10:58:39.000Z,...,2025-04-30 06:27:40.176749+08:00,False,False,0.802086,mixed,False,False,False,3.0,[AUTHOR] ayofvr [EMOJI] [CONTENT] Thank you G...
1,NaN,百達翡麗？ 沒有下限的網路病態！,16,Ch,2025年04月30日 06:27,Wednesday,6,morning,1,2025-04-29T12:33:44.000Z,...,2025-04-30 06:27:54.964788+08:00,True,True,0.673785,none,False,False,False,1.0,[EMOJI] [CONTENT] 百達翡麗？ 沒有下限的網路病態！
2,ribboworld2021,考完期中考，成績都還沒出來，小一女兒就自信的對我說：「我真羨慕妳生了一個天才！」,40,Ch,2025年04月30日 06:28,Wednesday,6,morning,0,2025-04-29T04:39:46.000Z,...,2025-04-30 06:28:09.873641+08:00,False,True,0.677911,1st,False,False,False,9.0,[AUTHOR] ribboworld2021 [TOPIC] 小一日常 [EMOJI] ...
3,ayofvr,Just be strong. Confident. Hopeful. Intellectu...,69,en,2025年04月30日 06:28,Wednesday,6,morning,0,2025-04-29T11:25:22.000Z,...,2025-04-30 06:28:24.726576+08:00,False,False,0.347473,1st,False,False,False,3.0,[AUTHOR] ayofvr [EMOJI] [CONTENT] Just be str...
4,jose_ykc,日文輔系老師上課內容之一AiScReam 歌詞導讀,25,Ch,2025年04月30日 06:28,Wednesday,6,morning,1,2025-04-29T10:36:19.000Z,...,2025-04-30 06:28:39.393706+08:00,False,False,0.164487,1st,False,False,False,3.0,[AUTHOR] jose_ykc [TOPIC] 輔仁大學 [EMOJI] [CONTE...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3096,leighton.williams,First tasting in California,27,en,2025年04月30日 00:23,Wednesday,0,night,0,2025年04月29日 07:30,...,2025-04-30 00:23:40.328046+08:00,False,False,0.447678,1st,False,False,False,8.0,[AUTHOR] leighton.williams [EMOJI] [CONTENT] ...
3097,cape__man,I hoped it would have been better.,34,en,2025年04月30日 00:23,Wednesday,0,night,0,2025年04月29日 06:31,...,2025-04-30 00:23:55.063517+08:00,False,False,0.149808,1st,False,False,False,9.0,[AUTHOR] cape__man [EMOJI] [CONTENT] I hoped ...
3098,simimoonlight,I can’t wait to watch Beyoncé on TikTok tonigh...,51,en,2025年04月30日 00:24,Wednesday,0,night,1,2025年04月29日 06:26,...,2025-04-30 00:24:39.870994+08:00,False,False,0.025393,1st,False,False,False,9.0,[AUTHOR] simimoonlight [EMOJI] 🫶 🏿 🥹 [CONTENT]...
3099,other98,"Tonight, Canada just proved that they have a h...",77,en,2025年04月30日 00:24,Wednesday,0,night,1,2025年04月29日 12:31,...,2025-04-30 00:24:54.669936+08:00,False,True,0.205699,mixed,False,False,False,3.0,"[AUTHOR] other98 [EMOJI] 🙌 [CONTENT] Tonight, ..."


In [ ]:
from transformers import BertTokenizer, BertModel
import torch
from torch.utils.data import Dataset, DataLoader

# 載入 BERT tokenizer 和模型
tokenizer = BertTokenizer.from_pretrained("bert-base-chinese")
model = BertModel.from_pretrained("bert-base-chinese")

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# 自訂 Dataset 類別（不變）
class TextDataset(Dataset):
    def __init__(self, texts, max_len=128):
        self.encodings = tokenizer(
            texts,
            truncation=True,
            padding='max_length',
            max_length=max_len,
            return_tensors="pt"
        )

    def __len__(self):
        return len(self.encodings["input_ids"])

    def __getitem__(self, idx):
        return {
            "input_ids": self.encodings["input_ids"][idx],
            "attention_mask": self.encodings["attention_mask"][idx]
        }

# 建立 dataset 和 dataloader（這裡建議用 semantic_text 而不是 content）
texts = df_semantic["semantic_text"].astype(str).tolist()
text_dataset = TextDataset(texts)
text_loader = DataLoader(text_dataset, batch_size=32)

# 提取語意向量 Z_text
Z_text_list = []

model.eval()
with torch.no_grad():
    for batch in text_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        cls_rep = outputs.last_hidden_state[:, 0, :]  # 取 [CLS] 向量

        Z_text_list.append(cls_rep.cpu())

Z_text_tensor = torch.cat(Z_text_list, dim=0)  # [N, 768]



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
import pandas as pd

Z_text_df = pd.DataFrame(Z_text_tensor.numpy())
Z_text_df

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,-1.197812,0.491429,-0.485847,0.316402,1.326632,-1.078312,0.297175,-0.804607,-1.177579,0.156056,...,-0.562811,-0.411341,0.433299,-0.422071,-0.669983,-0.726442,0.106794,1.173455,0.316311,0.108925
1,0.215058,1.000118,-0.044262,-0.209858,0.740915,-1.110935,-0.698523,-0.050961,-0.216489,1.010718,...,0.426819,-0.219904,-0.014253,-0.705548,0.285813,-0.596891,-0.212061,0.695831,-0.082648,0.383929
2,0.887392,0.952375,0.468289,0.363257,1.021098,-1.253422,-0.300589,-0.201750,-0.855972,0.126375,...,0.325380,-0.584822,-0.845941,-0.637422,-0.245426,0.500611,0.191020,0.498423,0.164274,-0.312768
3,-1.188988,0.500825,-1.228272,-0.300034,1.055789,-1.166171,0.346815,-0.645882,-0.857122,0.474148,...,-0.987928,-0.638471,-0.161923,-0.339970,-0.626446,-0.612481,0.194378,1.441311,-0.297321,0.020321
4,-0.932421,0.218100,-1.037626,0.035938,1.054755,-0.978883,-0.409518,0.187629,-0.958487,0.335751,...,-0.653079,-0.492126,0.483245,-0.070686,-0.069700,0.078444,0.408899,1.241559,-0.151853,-0.196576
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3078,-1.150746,0.211005,-0.857206,0.486619,0.887698,-1.112387,0.242535,-0.489165,-1.174779,0.081530,...,-0.239789,-0.378836,-0.029301,-0.404601,-0.395587,-0.504008,-0.001983,1.394138,-0.356525,-0.057205
3079,-0.742923,-0.134856,-0.337485,0.413056,1.205682,-1.366360,-0.096314,-0.656512,-1.172671,0.480091,...,-0.972686,-0.425607,0.257878,-0.082999,-0.208447,-0.519164,0.292847,1.117921,0.169527,0.234990
3080,-1.257780,0.298900,-0.860127,0.276141,1.046705,-0.745504,-0.342909,-0.462435,-1.735536,0.148791,...,-0.665856,-0.498026,-0.127218,-1.029894,-0.855349,-0.247065,0.405078,1.160196,0.243744,0.261610
3081,-0.801284,0.418369,-0.866499,-0.064372,0.974502,-1.084800,0.472064,-0.299250,-0.953438,0.027430,...,-1.174034,-0.653110,0.092793,-0.495103,-0.692929,-0.327009,0.045145,1.080404,0.078888,-0.075639


## MLP 處理數值資料

In [ ]:
import pandas as pd
import torch
import torch.nn as nn
from sklearn.preprocessing import StandardScaler

# 假設你已載入 DataFrame 為 df，並有這些欄位
numeric_cols = [
    "view_count", "followers_count", "emoji_count", "content_length", "post_hour", "time_elapsed_hours",
    "has_photo", "like_count", "reply_count", "repost_count", "share_count",
    "has_hashtag", "has_url", "has_mention", "has_exclaim", "has_question"
]

# 布林欄位轉 int（保險起見）
for col in numeric_cols:
    if df_semantic[col].dtype == bool:
        df_semantic[col] = df_semantic[col].astype(int)

# 標準化數值特徵
scaler = StandardScaler()
X_num_scaled = scaler.fit_transform(df_semantic[numeric_cols])

# 定義簡單的 MLP 模型
class NumericMLP(nn.Module):
    def __init__(self, input_dim, hidden_dim=64, output_dim=128):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(hidden_dim, output_dim)
        )

    def forward(self, x):
        return self.net(x)

# 初始化與執行模型
# 自動偵測你是否有 GPU（用 CUDA），否則就用 CPU
device = "cuda" if torch.cuda.is_available() else "cpu"
# 根據你數值欄位的數量設定 input_dim
model_num = NumericMLP(input_dim=X_num_scaled.shape[1])
# 把模型送到對應裝置（CPU / GPU）
model_num.to(device)

# 暫時關閉梯度運算，因沒有要訓練模型，僅讓資料過神經網路
with torch.no_grad():
    X_tensor = torch.tensor(X_num_scaled, dtype=torch.float32).to(device)
    Z_num_tensor = model_num(X_tensor).cpu()

# 合併 Z_text 和 Z_num
Z_text_tensor = torch.tensor(Z_text_df.values, dtype=torch.float32)  # [N, 768]
Z_full_tensor = torch.cat([Z_text_tensor, Z_num_tensor], dim=1)      # [N, 896]

print("數值向量與語意向量已成功融合為 Z_full_tensor")
Z_full_tensor

數值向量與語意向量已成功融合為 Z_full_tensor


tensor([[-1.1978e+00,  4.9143e-01, -4.8585e-01,  ...,  4.0996e-02,
         -1.0866e-02,  2.9819e-02],
        [ 2.1506e-01,  1.0001e+00, -4.4262e-02,  ...,  4.3622e-01,
          7.1429e-02,  2.5550e-01],
        [ 8.8739e-01,  9.5238e-01,  4.6829e-01,  ..., -1.5499e-01,
         -2.0740e-01,  5.4701e-01],
        ...,
        [-1.2578e+00,  2.9890e-01, -8.6013e-01,  ..., -1.0516e-03,
         -6.3461e-02,  9.6910e-02],
        [-8.0128e-01,  4.1837e-01, -8.6650e-01,  ...,  5.5146e-01,
         -1.4080e-01,  5.6089e-01],
        [-7.0800e-01,  6.7563e-01, -2.3394e-01,  ..., -1.5724e-01,
         -7.6307e-03,  3.6178e-01]])

## 清洗數據embbedding

In [ ]:
from datasets import Dataset

# =============== BERT 向量嵌入 ===============
df = df.dropna(subset=['content']) #要先處理content空值才能embedding
# --- 載入 tokenizer & model ---
tokenizer = AutoTokenizer.from_pretrained("bert-base-chinese")
model = AutoModel.from_pretrained("bert-base-chinese")
# 選擇硬體設備（MPS、CUDA、CPU），自動判斷是否可用 GPU（M1/M2 晶片上的 MPS 或 CUDA），否則 fallback 到 CPU
device = "mps" if torch.backends.mps.is_available() else "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# --- 建立 HuggingFace Dataset ---
hf_dataset = Dataset.from_pandas(df[["content"]])

# --- tokenize function ---
def tokenize_function(examples):
    return tokenizer(examples['content'], truncation=True, padding='max_length', max_length=128)
tokenized_dataset = hf_dataset.map(tokenize_function, batched=True)

# --- 取得 [CLS] 向量 ---
def extract_embeddings(batch):
    inputs = {k: torch.tensor(v).to(model.device) for k, v in batch.items() if k in tokenizer.model_input_names}
    with torch.no_grad():
        outputs = model(**inputs)
    embeddings = outputs.last_hidden_state[:, 0, :].detach().cpu().numpy()
    return {"embeddings": embeddings}

# --- 批次轉換為 embeddings ---
batch_size = 64
embeddings_dataset = tokenized_dataset.map(extract_embeddings, batched=True, batch_size=batch_size)

# =============== 匯出最終結果 ===============
# embeddings_dataset["embeddings"] 是 list of 768-dim vectors
embedding_df = pd.DataFrame(embeddings_dataset["embeddings"])
final_df = pd.concat([df.reset_index(drop=True), embedding_df], axis=1)

# 儲存
# final_df.to_csv("C:/Users/User/Desktop/louis/threads_with_embeddings.csv",encoding='utf_8_sig', index=False)
final_df.to_csv("threads_with_embeddings.csv",encoding='utf_8_sig', index=False)
print("✅ 全部處理完成，已輸出 threads_with_embeddings.csv")

Map:   0%|          | 0/3083 [00:00<?, ? examples/s]

Map:   0%|          | 0/3083 [00:00<?, ? examples/s]

✅ 全部處理完成，已輸出 threads_with_embeddings.csv


## 分詞

In [ ]:
stopwords = set(['的', '了', '在', '是', '和', '也', '與', '有', '為', '等'])

def tokenize_and_remove_stopwords(text):
    if not isinstance(text, str):
        return ''
    words = jieba.cut(text)
    words_filtered = [word for word in words if word not in stopwords]
    return ' '.join(words_filtered)


df['processed_content'] = df['content'].apply(tokenize_and_remove_stopwords)
df['processed_content'][2]

Building prefix dict from the default dictionary ...
DEBUG:jieba:Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
DEBUG:jieba:Loading model from cache /tmp/jieba.cache
Loading model cost 0.819 seconds.
DEBUG:jieba:Loading model cost 0.819 seconds.
Prefix dict has been built successfully.
DEBUG:jieba:Prefix dict has been built successfully.


'考完 期中考 ， 成績 都 還沒出 來 ， 小一 女兒 就 自信 對 我 說 ： 「 我 真羨慕 妳 生 一個 天才 ！ 」'

## 機器學習建模

In [ ]:
# 計算 TF-IDF
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(df['processed_content'])

# 計算 TF
tf_vectorizer = CountVectorizer()
tf_matrix = tf_vectorizer.fit_transform(df['processed_content'])

print(tfidf_matrix)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 33538 stored elements and shape (3083, 12873)>
  Coords	Values
  (0, 2799)	0.46630568431527075
  (0, 3188)	0.2940455207159238
  (0, 1306)	0.442840976271973
  (0, 1210)	0.31028579706458703
  (0, 412)	0.5031213376250259
  (0, 892)	0.3880460720366237
  (1, 9695)	0.44774054192975266
  (1, 10472)	0.44774054192975266
  (1, 8813)	0.2568875443528821
  (1, 3491)	0.42597495668833046
  (1, 10367)	0.38876644295524665
  (1, 9613)	0.44774054192975266
  (2, 10514)	0.31144203947399207
  (2, 8415)	0.3288785193471119
  (2, 7290)	0.28537598711095685
  (2, 12071)	0.35868637857163266
  (2, 6458)	0.35868637857163266
  (2, 6044)	0.2990706601225911
  (2, 10681)	0.3288785193471119
  (2, 9846)	0.35868637857163266
  (2, 3221)	0.18120453315645962
  (2, 5960)	0.31144203947399207
  (3, 1628)	0.2616426750239415
  (3, 538)	0.23056465830230902
  (3, 2699)	0.37286988409222926
  :	:
  (3080, 3034)	0.4140307205152917
  (3080, 3064)	0.38762922360836444
  (3080,

# 多模型分類實驗

In [ ]:
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler, random_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report
from transformers import AutoTokenizer, AutoModel
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
batch_size = 8
num_epochs = 5
# ========== 參數設定 ==========
model_tokenizer_map = {
    "FusionMacBERT": "hfl/chinese-macbert-base",
    "PureMacBERT": "hfl/chinese-macbert-base",
    "NumericOnly": None,
    "BiLSTMWithNumeric": "bert-base-chinese",
    "MacBERTWithGRU": "hfl/chinese-macbert-base",
    "MacBERTMLPFusion": "hfl/chinese-macbert-base",
    "TextCNNMacBERT": "hfl/chinese-macbert-base",
    "RoBERTa": "hfl/chinese-roberta-wwm-ext",
    "BERTwwmExt": "hfl/chinese-bert-wwm-ext",
    "ERNIE": "nghuyong/ernie-3.0-base-zh",
    "ConvBERT": "YituTech/conv-bert-base"
}

#tokenizer
default_tokenizer_name = model_tokenizer_map["FusionMacBERT"]
tokenizer = AutoTokenizer.from_pretrained(default_tokenizer_name)

#載入資料
# df = pd.read_csv("C:/Users/User/Desktop/louis/threads_cleaned_v2.csv", encoding='utf_8_sig')
#df = df.dropna(subset=['content', 'view_count']).reset_index(drop=True)
df

,author,content,content_length,lang,scrape_time,post_weekday,post_hour,post_period,viral,post_time,...,scrape_time_origin,has_question,has_exclaim,sentiment_score,person_view,has_url,has_mention,has_hashtag,time_elapsed_hours,processed_content
0,ayofvr,Thank you God for another day.,30,en,2025年04月30日 06:27,Wednesday,6,morning,0,2025-04-29T10:58:39.000Z,...,2025-04-30 06:27:40.176749+08:00,False,False,0.802086,mixed,False,False,False,3.0,Thank you God for another day .
1,NaN,百達翡麗？ 沒有下限的網路病態！,16,Ch,2025年04月30日 06:27,Wednesday,6,morning,1,2025-04-29T12:33:44.000Z,...,2025-04-30 06:27:54.964788+08:00,True,True,0.673785,none,False,False,False,1.0,百達 翡麗 ？ 沒有 下限 網路 病態 ！
2,ribboworld2021,考完期中考，成績都還沒出來，小一女兒就自信的對我說：「我真羨慕妳生了一個天才！」,40,Ch,2025年04月30日 06:28,Wednesday,6,morning,0,2025-04-29T04:39:46.000Z,...,2025-04-30 06:28:09.873641+08:00,False,True,0.677911,1st,False,False,False,9.0,考完 期中考 ， 成績 都 還沒出 來 ， 小一 女兒 就 自信 對 我 說 ： 「 我 真...
3,ayofvr,Just be strong. Confident. Hopeful. Intellectu...,69,en,2025年04月30日 06:28,Wednesday,6,morning,0,2025-04-29T11:25:22.000Z,...,2025-04-30 06:28:24.726576+08:00,False,False,0.347473,1st,False,False,False,3.0,Just be strong . Confident . Hopeful ....
4,jose_ykc,日文輔系老師上課內容之一AiScReam 歌詞導讀,25,Ch,2025年04月30日 06:28,Wednesday,6,morning,1,2025-04-29T10:36:19.000Z,...,2025-04-30 06:28:39.393706+08:00,False,False,0.164487,1st,False,False,False,3.0,日文 輔系 老師 上 課內容 之一 AiScReam 歌詞 導讀
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3096,leighton.williams,First tasting in California,27,en,2025年04月30日 00:23,Wednesday,0,night,0,2025年04月29日 07:30,...,2025-04-30 00:23:40.328046+08:00,False,False,0.447678,1st,False,False,False,8.0,First tasting in California
3097,cape__man,I hoped it would have been better.,34,en,2025年04月30日 00:23,Wednesday,0,night,0,2025年04月29日 06:31,...,2025-04-30 00:23:55.063517+08:00,False,False,0.149808,1st,False,False,False,9.0,I hoped it would have been better .
3098,simimoonlight,I can’t wait to watch Beyoncé on TikTok tonigh...,51,en,2025年04月30日 00:24,Wednesday,0,night,1,2025年04月29日 06:26,...,2025-04-30 00:24:39.870994+08:00,False,False,0.025393,1st,False,False,False,9.0,I can ’ t wait to watch Beyonc é o...
3099,other98,"Tonight, Canada just proved that they have a h...",77,en,2025年04月30日 00:24,Wednesday,0,night,1,2025年04月29日 12:31,...,2025-04-30 00:24:54.669936+08:00,False,True,0.205699,mixed,False,False,False,3.0,"Tonight , Canada just proved that th..."


# Label 分群 (用四分位數分三群)

In [ ]:
'''
# Label 分群 ：標籤轉換（按瀏覽數進行分群）
# 取「瀏覽數」的第 80 百分位作為高人氣門檻（q_high）、第 20 百分位作為低人氣門檻（q_low）
# 把每筆資料的「view_count」劃分為三類：0 高人氣 (high)、1 中人氣 (medium)、2 低人氣 (low)
q_high = df['view_count'].quantile(0.80)
q_low = df['view_count'].quantile(0.20)
df['view_class'] = df['view_count'].apply(lambda x: "high" if x >= q_high else ("low" if x <= q_low else "medium"))
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['view_class'])

#做 oversampling 類別資料平衡 ：資料增強（針對 high / low 類別 oversample）
df_high = df[df['view_class'] == 'high']
df_low = df[df['view_class'] == 'low']
df_medium = df[df['view_class'] == 'medium']

# 分別取出三個分類的樣本：對 high 與 low 分類做「過採樣」，各自複製三次，讓資料數量接近 medium
# 再對整個資料表做隨機打散 (shuffle），避免模型學到資料順序的偏誤
df_high_oversampled = pd.concat([df_high] * 3, ignore_index=True)
df_low_oversampled = pd.concat([df_low] * 3, ignore_index=True)
df = pd.concat([df_medium, df_high_oversampled, df_low_oversampled], ignore_index=True)
df = df.sample(frac=1).reset_index(drop=True)
'''

def map_view_class(x):
    if x < 1000:
        return 'low'
    elif x < 10000:
        return 'medium'
    elif x < 100000:
        return 'high'
    else:
        return 'very_high'

df['view_class'] = df['view_count'].apply(map_view_class)
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['view_class'])
df_high = df[df['view_class'] == 'very_high']
df_medium = df[df['view_class'] == 'high']
df_low = df[df['view_class'] == 'medium']
df_very_low = df[df['view_class'] == 'low']

df_high_oversampled = pd.concat([df_high] * 3, ignore_index=True)
df_very_low_oversampled = pd.concat([df_very_low] * 3, ignore_index=True)

df = pd.concat([df_medium, df_low, df_high_oversampled, df_very_low_oversampled], ignore_index=True)
df = df.sample(frac=1).reset_index(drop=True)


In [ ]:
q_high

np.float64(5825.600000000002)

In [ ]:
q_low

np.float64(248.4000000000001)

## Label 分群 （1000以下、1000~10000、10000~100000、100000以上)

In [ ]:

# 將 view_count 分成四類：
# 0: 小於 1000
# 1: 1000 ~ 9999
# 2: 10000 ~ 99999
# 3: 100000 以上

def map_view_class(x):
    if x < 1000:
        return 'low'
    elif x < 10000:
        return 'medium'
    elif x < 100000:
        return 'high'
    else:
        return 'very_high'

df['view_class'] = df['view_count'].apply(map_view_class)

# 編碼成數字 label
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['view_class'])

df_high = df[df['view_class'] == 'very_high']
df_medium = df[df['view_class'] == 'high']
df_low = df[df['view_class'] == 'medium']
df_very_low = df[df['view_class'] == 'low']

# 針對較少的類別進行擴增（假設 high 和 very_low 比較少）
df_high_oversampled = pd.concat([df_high] * 3, ignore_index=True)
df_very_low_oversampled = pd.concat([df_very_low] * 3, ignore_index=True)

# 合併並打亂
df = pd.concat([df_medium, df_low, df_high_oversampled, df_very_low_oversampled], ignore_index=True)
df = df.sample(frac=1).reset_index(drop=True)

df



,author,content,content_length,lang,scrape_time,post_weekday,post_hour,post_period,viral,post_time,...,has_exclaim,sentiment_score,person_view,has_url,has_mention,has_hashtag,time_elapsed_hours,processed_content,view_class,label
0,iamverafaith,有人今晚六點從員林出發去白沙屯的勇腳嗎？,20,Ch,2025年05月02日 01:36,Friday,1,night,0,2025-05-01T07:41:55.000Z,...,False,0.923232,none,False,False,False,1.00,有人 今晚 六點 從員林出 發去 白沙 屯 勇腳 嗎 ？,medium,2
1,kjybecauseisaidso,mehhhhhh,8,bs,2025年04月28日 04:40,Monday,4,night,0,2025年04月27日 03:34,...,False,0.500000,none,False,False,False,17.00,mehhhhhh,medium,2
2,NaN,On site for Game 6 😤 @rakuten|| #DubNation,42,en,2025年05月03日 19:56,Saturday,19,evening,1,2025-05-02T23:23:35.000Z,...,False,0.253942,1st,False,True,True,4.00,On site for Game 6 😤 @ rakuten | |...,high,0
3,monicascassi,Booking Beyoncé tickets just to see blue ivy a...,53,en,2025年05月03日 08:36,Saturday,8,morning,1,2025-05-02T14:14:58.000Z,...,True,0.070694,1st,False,False,False,2.00,Booking Beyonc é tickets just to see...,high,0
4,ions_0712,我等妳 妳不要丟下我好不好 4,15,Ch,2025年05月03日 19:38,Saturday,19,evening,0,2025-05-03T02:25:17.000Z,...,False,0.087091,1st,False,False,False,1.00,我 妳 妳 不要 丟下 我 好不好 4,medium,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11342,er1dachi,This game been off and fed from an off energy ...,201,en,2025年05月03日 20:37,Saturday,20,evening,0,2025-05-03T03:38:45.000Z,...,False,0.001674,mixed,False,False,False,0.95,This game been off and fed from ...,low,1
11343,love_karina1418,昨天夢到我走在路上好像穿著海嫄的衣服 然後海嫄本人過來幫我簽名🥹然後我就起床了,39,Ch,2025年04月29日 02:31,Tuesday,2,night,0,2025年04月28日 13:54,...,False,0.212831,1st,False,False,False,4.00,昨天 夢到 我 走 路上 好像 穿著 海嫄 衣服 然後海 嫄 本人 過來 幫 我 簽名 ...,low,1
11344,c8763_fk,最讓人傷心的事情 是你有一群朋友喜歡玩一款你不怎麼喜歡玩的遊戲而你為了和他們一起玩只能玩著自...,161,Ch,2025年05月03日 18:57,Saturday,18,evening,0,2025-05-02T15:35:18.000Z,...,False,0.010719,mixed,False,False,False,11.00,最 讓 人 傷心 事情 你 一群 朋友 喜歡 玩 一款 你 不 怎麼 喜歡 玩 遊戲 而...,low,1
11345,_2156_e,開心 收到刷卡通知還以為被盜刷 升席成功✅拜託日巡一直繼續對我好🤲🏻🍀,35,Ch,2025年04月30日 17:22,Wednesday,17,evening,0,2025-04-29T18:04:36.000Z,...,False,0.086483,1st,False,False,False,7.00,開心 收到 刷卡 通知 還以 被 盜 刷 升席 成功 ✅ 拜 託 日 巡 一直 繼續...,low,1


In [ ]:
# 

In [ ]:

from sklearn.preprocessing import LabelEncoder

#加入情緒與人稱特徵欄位
df["person_view_encoded"] = LabelEncoder().fit_transform(df["person_view"])
if "sentiment_score" not in df.columns:
    df["sentiment_score"] = 0.5


# 加入數值特徵欄
df["sentiment_label_encoded"] = LabelEncoder().fit_transform(df["sentiment_label"])


#更新數值特徵欄位 ===
num_cols = ["like_count", "reply_count", "emoji_count", "sentiment_score", "sentiment_label_encoded","person_view_encoded"]


In [ ]:
class RegressionDataset(Dataset):
    def __init__(self, df):
        self.input_ids = df['input_ids'].tolist()
        self.attention_mask = df['attention_mask'].tolist()
        self.numerics = df[num_cols].values
        self.targets = df['target'].values

    def __len__(self):
        return len(self.targets)

    def __getitem__(self, idx):
        return {
            'input_ids': torch.tensor(self.input_ids[idx], dtype=torch.long),
            'attention_mask': torch.tensor(self.attention_mask[idx], dtype=torch.long),
            'numerics': torch.tensor(self.numerics[idx], dtype=torch.float),
            'target': torch.tensor(self.targets[idx], dtype=torch.float)
        }


# Normalization 數值特徵標準化

In [ ]:
# Normalization 數值特徵標準化
base_num_cols = ['like_count', 'share_count', 'repost_count', 'reply_count', 'emoji_count', 'has_photo', 'has_video', 'has_question', 'has_exclaim', 'has_mention', 'has_url', 'has_hashtag', 'content_length']
# 找出 one-hot 編碼的欄位（語言類型、發文時段、星期幾等類別欄位）
# 使用 StandardScaler 將數值欄位轉換為「標準常態分布」（mean=0, std=1），有助於模型學習穩定。
onehot_cols = [col for col in df.columns if col.startswith('lang_') or col.startswith('post_period_') or col.startswith('post_weekday_')]
num_cols = base_num_cols + onehot_cols
scaler = StandardScaler()
df[num_cols] = scaler.fit_transform(df[num_cols])

# Tokenizer 文本編碼 ：使用事先定義好的 tokenizer（例如 MacBERT、RoBERTa）對貼文進行斷詞、編碼
# 將編碼後的結果儲存到 df 中，這兩個欄位會作為 BERT 模型的輸入
encodings = tokenizer(df['content'].tolist(), truncation=True, padding='max_length', max_length=128)
df['input_ids'] = encodings['input_ids']  # 斷詞後對應的詞彙 ID
df['attention_mask'] = encodings['attention_mask']  # 對應位置是否是 padding（0）或實際內容（1）

In [ ]:
df

,author,content,content_length,lang,scrape_time,post_weekday,post_hour,post_period,viral,post_time,...,person_view,has_url,has_mention,has_hashtag,time_elapsed_hours,processed_content,view_class,label,input_ids,attention_mask
0,iamverafaith,有人今晚六點從員林出發去白沙屯的勇腳嗎？,-0.564386,Ch,2025年05月02日 01:36,Friday,1,night,0,2025-05-01T07:41:55.000Z,...,none,-0.028174,-0.152533,-0.153739,1.00,有人 今晚 六點 從員林出 發去 白沙 屯 勇腳 嗎 ？,medium,2,"[101, 3300, 782, 791, 3241, 1063, 7953, 2537, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,kjybecauseisaidso,mehhhhhh,-0.783290,bs,2025年04月28日 04:40,Monday,4,night,0,2025年04月27日 03:34,...,none,-0.028174,-0.152533,-0.153739,17.00,mehhhhhh,medium,2,"[101, 8450, 8199, 8199, 8199, 8199, 8199, 8199...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, ..."
2,NaN,On site for Game 6 😤 @rakuten|| #DubNation,-0.163063,en,2025年05月03日 19:56,Saturday,19,evening,1,2025-05-02T23:23:35.000Z,...,1st,-0.028174,6.555961,6.504549,4.00,On site for Game 6 😤 @ rakuten | |...,high,0,"[101, 8281, 11215, 8330, 9033, 127, 100, 137, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,monicascassi,Booking Beyoncé tickets just to see blue ivy a...,0.037598,en,2025年05月03日 08:36,Saturday,8,morning,1,2025-05-02T14:14:58.000Z,...,1st,-0.028174,-0.152533,-0.153739,2.00,Booking Beyonc é tickets just to see...,high,0,"[101, 9309, 8815, 10222, 9056, 9654, 10811, 81...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,ions_0712,我等妳 妳不要丟下我好不好 4,-0.655596,Ch,2025年05月03日 19:38,Saturday,19,evening,0,2025-05-03T02:25:17.000Z,...,1st,-0.028174,-0.152533,-0.153739,1.00,我 妳 妳 不要 丟下 我 好不好 4,medium,2,"[101, 2769, 5023, 1986, 1986, 679, 6206, 694, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11342,er1dachi,This game been off and fed from an off energy ...,2.737407,en,2025年05月03日 20:37,Saturday,20,evening,0,2025-05-03T03:38:45.000Z,...,mixed,-0.028174,-0.152533,-0.153739,0.95,This game been off and fed from ...,low,1,"[101, 8554, 9033, 8815, 8329, 9594, 8256, 9311...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
11343,love_karina1418,昨天夢到我走在路上好像穿著海嫄的衣服 然後海嫄本人過來幫我簽名🥹然後我就起床了,-0.217789,Ch,2025年04月29日 02:31,Tuesday,2,night,0,2025年04月28日 13:54,...,1st,-0.028174,-0.152533,-0.153739,4.00,昨天 夢到 我 走 路上 好像 穿著 海嫄 衣服 然後海 嫄 本人 過來 幫 我 簽名 ...,low,1,"[101, 3219, 1921, 1918, 1168, 2769, 6624, 1762...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
11344,c8763_fk,最讓人傷心的事情 是你有一群朋友喜歡玩一款你不怎麼喜歡玩的遊戲而你為了和他們一起玩只能玩著自...,2.007729,Ch,2025年05月03日 18:57,Saturday,18,evening,0,2025-05-02T15:35:18.000Z,...,mixed,-0.028174,-0.152533,-0.153739,11.00,最 讓 人 傷心 事情 你 一群 朋友 喜歡 玩 一款 你 不 怎麼 喜歡 玩 遊戲 而...,low,1,"[101, 3297, 6366, 782, 1003, 2552, 4638, 752, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
11345,_2156_e,開心 收到刷卡通知還以為被盜刷 升席成功✅拜託日巡一直繼續對我好🤲🏻🍀,-0.290757,Ch,2025年04月30日 17:22,Wednesday,17,evening,0,2025-04-29T18:04:36.000Z,...,1st,-0.028174,-0.152533,-0.153739,7.00,開心 收到 刷卡 通知 還以 被 盜 刷 升席 成功 ✅ 拜 託 日 巡 一直 繼續...,low,1,"[101, 7274, 2552, 3119, 1168, 1170, 1305, 6858...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


In [ ]:
# Dataset
class CustomDataset(Dataset):
    def __init__(self, df):
        self.input_ids = df['input_ids'].tolist()
        self.attention_mask = df['attention_mask'].tolist()
        self.labels = df['label'].tolist()
        self.numerics = df[num_cols].values

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            'input_ids': torch.tensor(self.input_ids[idx], dtype=torch.long),
            'attention_mask': torch.tensor(self.attention_mask[idx], dtype=torch.long),
            'numerics': torch.tensor(self.numerics[idx], dtype=torch.float),
            'labels': torch.tensor(self.labels[idx], dtype=torch.long)
        }

# Loss
class FocalLoss(nn.Module):
    def __init__(self, gamma=2.0):
        super().__init__()
        self.gamma = gamma
        self.ce = nn.CrossEntropyLoss()

    def forward(self, input, target):
        logp = self.ce(input, target)
        p = torch.exp(-logp)
        loss = (1 - p) ** self.gamma * logp
        return loss.mean()

# 模型架構

In [ ]:
#模型架構
# 1. FusionMacBERT：BERT + 數值特徵 concat
class FusionMacBERTModel(nn.Module):
    def __init__(self, model_name, num_numeric_features, num_classes):
        super().__init__()
        self.bert = AutoModel.from_pretrained(model_name)
        self.dropout = nn.Dropout(0.3)
        self.numeric_fc = nn.Linear(num_numeric_features, 64)
        self.classifier = nn.Linear(self.bert.config.hidden_size + 64, num_classes)

    def forward(self, input_ids, attention_mask, numerics):
        cls_output = self.bert(input_ids=input_ids, attention_mask=attention_mask).last_hidden_state[:, 0, :]
        num_out = torch.relu(self.numeric_fc(numerics))
        combined = torch.cat((cls_output, num_out), dim=1)
        return self.classifier(self.dropout(combined))

# 2. PureMacBERT：只有文字
class PureMacBERTModel(nn.Module):
    def __init__(self, model_name, num_classes):
        super().__init__()
        self.bert = AutoModel.from_pretrained(model_name)
        self.dropout = nn.Dropout(0.3)
        self.classifier = nn.Linear(self.bert.config.hidden_size, num_classes)

    def forward(self, input_ids, attention_mask, numerics=None):
        cls_output = self.bert(input_ids=input_ids, attention_mask=attention_mask).last_hidden_state[:, 0, :]
        return self.classifier(self.dropout(cls_output))

# 3. NumericOnly：只有數值特徵
class NumericOnlyModel(nn.Module):
    def __init__(self, num_numeric_features, num_classes):
        super().__init__()
        self.classifier = nn.Sequential(
            nn.Linear(num_numeric_features, 64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, num_classes)
        )

    def forward(self, input_ids=None, attention_mask=None, numerics=None):
        return self.classifier(numerics)

# 4. BiLSTMWithNumeric：LSTM 處理詞嵌入 + 數值特徵
class BiLSTMWithNumeric(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_numeric_features, num_classes):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True, bidirectional=True)
        self.numeric_fc = nn.Linear(num_numeric_features, 64)
        self.dropout = nn.Dropout(0.3)
        self.classifier = nn.Linear(hidden_dim * 2 + 64, num_classes)

    def forward(self, input_ids, attention_mask, numerics):
        x = self.embedding(input_ids)
        lstm_out, _ = self.lstm(x)
        pooled = lstm_out[:, -1, :]
        num_out = torch.relu(self.numeric_fc(numerics))
        combined = torch.cat((pooled, num_out), dim=1)
        return self.classifier(self.dropout(combined))

# 5. MacBERTWithGRU：BERT + GRU + 數值特徵
class MacBERTWithGRU(nn.Module):
    def __init__(self, model_name, num_numeric_features, num_classes):
        super().__init__()
        self.bert = AutoModel.from_pretrained(model_name)
        self.gru = nn.GRU(self.bert.config.hidden_size, 128, batch_first=True, bidirectional=True)
        self.numeric_fc = nn.Linear(num_numeric_features, 64)
        self.classifier = nn.Linear(128*2 + 64, num_classes)
        self.dropout = nn.Dropout(0.3)

    def forward(self, input_ids, attention_mask, numerics):
        bert_out = self.bert(input_ids=input_ids, attention_mask=attention_mask).last_hidden_state
        gru_out, _ = self.gru(bert_out)
        pooled = gru_out[:, -1, :]
        num_out = torch.relu(self.numeric_fc(numerics))
        combined = torch.cat((pooled, num_out), dim=1)
        return self.classifier(self.dropout(combined))

# 6. MacBERTMLPFusion：BERT + 數值特徵 -> MLP
class MacBERTMLPFusion(nn.Module):
    def __init__(self, model_name, num_numeric_features, num_classes):
        super().__init__()
        self.bert = AutoModel.from_pretrained(model_name)
        self.fc = nn.Sequential(
            nn.Linear(self.bert.config.hidden_size + num_numeric_features, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, num_classes)
        )

    def forward(self, input_ids, attention_mask, numerics):
        cls_output = self.bert(input_ids=input_ids, attention_mask=attention_mask).last_hidden_state[:, 0, :]
        combined = torch.cat((cls_output, numerics), dim=1)
        return self.fc(combined)

# 7. TextCNNMacBERT：BERT 輸出卷積後 + 數值特徵
class TextCNNMacBERT(nn.Module):
    def __init__(self, model_name, num_numeric_features, num_classes):
        super().__init__()
        self.bert = AutoModel.from_pretrained(model_name)
        self.convs = nn.ModuleList([nn.Conv2d(1, 64, (k, self.bert.config.hidden_size)) for k in [2, 3, 4]])
        self.numeric_fc = nn.Linear(num_numeric_features, 64)
        self.classifier = nn.Linear(64 * len([2, 3, 4]) + 64, num_classes)

    def conv_and_pool(self, x, conv):
        x = torch.relu(conv(x)).squeeze(3)
        x = torch.max_pool1d(x, x.size(2)).squeeze(2)
        return x

    def forward(self, input_ids, attention_mask, numerics):
        x = self.bert(input_ids=input_ids, attention_mask=attention_mask).last_hidden_state.unsqueeze(1)
        cnn_out = torch.cat([self.conv_and_pool(x, conv) for conv in self.convs], 1)
        num_out = torch.relu(self.numeric_fc(numerics))
        combined = torch.cat((cnn_out, num_out), dim=1)
        return self.classifier(combined)


In [ ]:
#訓練與評估
def train_and_eval(model, name, preview_count=10):
    model = model.to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))
    optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)

    loss_fn = FocalLoss()
    # loss_fn = nn.MSELoss()

    # 訓練階段
    for epoch in range(5):
        model.train()
        for batch in train_loader:
            optimizer.zero_grad()
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            numerics = batch['numerics'].to(device)
            labels = batch['labels'].to(device)
            output = model(input_ids, attention_mask, numerics)
            loss = loss_fn(output, labels)
            loss.backward()
            optimizer.step()

    # 評估階段
    model.eval()
    all_preds, all_labels = [], []
    '''
    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            numerics = batch['numerics'].to(device)
            labels = batch['labels'].to(device)
            output = model(input_ids, attention_mask, numerics)
            preds = output.argmax(dim=1).cpu().numpy()
            all_preds.extend(preds)
            all_labels.extend(labels.cpu().numpy())

    from sklearn.metrics import mean_squared_error, mean_absolute_error
    mse = mean_squared_error(all_targets, all_preds)
    mae = mean_absolute_error(all_targets, all_preds)
    print(f"[{name} 評估結果] MSE: {mse:.2f} | MAE: {mae:.2f}")
    print(f"\n{name} 評估結果：")
    print(classification_report(all_labels, all_preds, target_names=label_encoder.classes_))
    '''
    preview_shown = 0
    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            numerics = batch['numerics'].to(device)
            labels = batch['labels'].to(device)
            output = model(input_ids, attention_mask, numerics)
            preds = output.argmax(dim=1).cpu().numpy()
            all_preds.extend(preds)
            all_labels.extend(labels.cpu().numpy())

            #印出前幾筆的預測、真實值
            if preview_shown < preview_count:
                batch_size = input_ids.shape[0]
                for i in range(batch_size):
                    if preview_shown >= preview_count:
                        break
                    input_id = input_ids[i].cpu().numpy()
                    text = tokenizer.decode(input_id, skip_special_tokens=True)
                    print(f"\n[{name} 預測] 第 {preview_shown+1} 筆")
                    print(f"Text: {text}")
                    print(f"Predicted: {label_encoder.inverse_transform([preds[i]])[0]}")
                    print(f"Actual:    {label_encoder.inverse_transform([labels[i].cpu().item()])[0]}")
                    preview_shown += 1

    print(f"\n{name} 評估結果：")
    print(classification_report(all_labels, all_preds, target_names=label_encoder.classes_))

# 跑模型

In [ ]:
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import DataLoader, WeightedRandomSampler, random_split

#轉換label
df = df.dropna(subset=["content", "label"]).reset_index(drop=True)
df["label"] = LabelEncoder().fit_transform(df["label"])
num_classes = df["label"].nunique()



#Tokenizer
tokenizer = AutoTokenizer.from_pretrained("hfl/chinese-macbert-base")  # 預設 tokenizer，可換
encodings = tokenizer(df['content'].tolist(), truncation=True, padding='max_length', max_length=128)
df['input_ids'] = encodings['input_ids']
df['attention_mask'] = encodings['attention_mask']

#Dataset
dataset = CustomDataset(df)
train_size = int(0.8 * len(dataset))
train_dataset, test_dataset = random_split(dataset, [train_size, len(dataset)-train_size])

train_labels = [train_dataset[i]['labels'].item() for i in range(len(train_dataset))]
class_counts = pd.Series(train_labels).value_counts().to_dict()
weights = [1.0 / class_counts[label] for label in train_labels]
sampler = WeightedRandomSampler(weights, len(weights), replacement=True)

train_loader = DataLoader(train_dataset, batch_size=8, sampler=sampler)
test_loader = DataLoader(test_dataset, batch_size=8)

#模型定義
model_variants = {
    "FusionMacBERT": FusionMacBERTModel("hfl/chinese-macbert-base", len(num_cols), num_classes),
    "PureMacBERT": PureMacBERTModel("hfl/chinese-macbert-base", num_classes),
    "NumericOnly": NumericOnlyModel(len(num_cols), num_classes),
    "BiLSTMWithNumeric": BiLSTMWithNumeric(tokenizer.vocab_size, 128, 128, len(num_cols), num_classes),
    "MacBERTWithGRU": MacBERTWithGRU("hfl/chinese-macbert-base", len(num_cols), num_classes),
    "MacBERTMLPFusion": MacBERTMLPFusion("hfl/chinese-macbert-base", len(num_cols), num_classes),
    "TextCNNMacBERT": TextCNNMacBERT("hfl/chinese-macbert-base", len(num_cols), num_classes),
    "RoBERTa": FusionMacBERTModel("hfl/chinese-roberta-wwm-ext", len(num_cols), num_classes),
    "BERTwwmExt": FusionMacBERTModel("hfl/chinese-bert-wwm-ext", len(num_cols), num_classes),
    "ERNIE": FusionMacBERTModel("nghuyong/ernie-3.0-base-zh", len(num_cols), num_classes),
    "ConvBERT": FusionMacBERTModel("YituTech/conv-bert-base", len(num_cols), num_classes)
}

#模型訓練
for name, model in model_variants.items():
    print(f"訓練模型: {name}")
    train_and_eval(model, name)


訓練模型: FusionMacBERT


KeyboardInterrupt: 

In [ ]:
'''
# 資料分割：資料集切分與取樣
dataset = CustomDataset(df)
train_size = int(0.8 * len(dataset))
train_dataset, test_dataset = random_split(dataset, [train_size, len(dataset)-train_size])
train_labels = [train_dataset[i]['labels'].item() for i in range(len(train_dataset))]
class_counts = pd.Series(train_labels).value_counts().to_dict()
weights = [1.0 / class_counts[label] for label in train_labels]
sampler = WeightedRandomSampler(weights, len(weights), replacement=True)
train_loader = DataLoader(train_dataset, batch_size=8, sampler=sampler)
test_loader = DataLoader(test_dataset, batch_size=8)

# 執行多模型訓練
model_variants = {
    #"FusionMacRegressor": FusionMacBERTRegressor("hfl/chinese-macbert-base", len(num_cols), 3),
    "FusionMacBERT": FusionMacBERTModel("hfl/chinese-macbert-base", len(num_cols), 3),
    "PureMacBERT": PureMacBERTModel("hfl/chinese-macbert-base", 3),
    "NumericOnly": NumericOnlyModel(len(num_cols), 3),
    "BiLSTMWithNumeric": BiLSTMWithNumeric(tokenizer.vocab_size, 128, 128, len(num_cols), 3),
    "MacBERTWithGRU": MacBERTWithGRU("hfl/chinese-macbert-base", len(num_cols), 3),
    "MacBERTMLPFusion": MacBERTMLPFusion("hfl/chinese-macbert-base", len(num_cols), 3),
    "TextCNNMacBERT": TextCNNMacBERT("hfl/chinese-macbert-base", len(num_cols), 3),
    "RoBERTa": FusionMacBERTModel("hfl/chinese-roberta-wwm-ext", len(num_cols), 3),
    "BERTwwmExt": FusionMacBERTModel("hfl/chinese-bert-wwm-ext", len(num_cols), 3),
    "ERNIE": FusionMacBERTModel("nghuyong/ernie-3.0-base-zh", len(num_cols), 3),
    "ConvBERT": FusionMacBERTModel("YituTech/conv-bert-base", len(num_cols), 3)
}

# 逐個模型訓練與輸出結果
for name, model in model_variants.items():
    tokenizer_name = model_tokenizer_map.get(name, default_tokenizer_name)
    if tokenizer_name:
        tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
        encodings = tokenizer(df['content'].tolist(), truncation=True, padding='max_length', max_length=128)
        df['input_ids'] = encodings['input_ids']
        df['attention_mask'] = encodings['attention_mask']
    train_and_eval(model, name)
'''

1. FusionMacBERT ✅文字 + ✅數值
BERT： 使用 MacBERT
架構： 把 [CLS] 向量與數值特徵經過 MLP 融合
用途： 做為 baseline 融合模型
優點： 同時考慮內容語義與貼文統計資料（如按讚數、是否有 hashtag）

2. PureMacBERT ✅文字 + ❌數值
BERT： 使用 MacBERT
架構： 單純使用 [CLS]，後接 linear 層分類
用途： 純語言模型 baseline
對照： 可用來比較是否有數值輔助提升效果

3. NumericOnly ❌文字 + ✅數值
模型類型： 只有數值輸入，經過 MLP 做分類
用途： 測試「只靠貼文統計資料」能否達到合理分類
對照： 可與文字模型或融合模型對比效果

4. BiLSTMWithNumeric ✅文字（Embedding+LSTM）+ ✅數值
嵌入方式： 使用 nn.Embedding + BiLSTM 處理文字（不是 BERT）
融合方式： 將 LSTM 最後時間步 + 數值特徵拼接
特別點： 測試「非 Transformer 模型」是否仍具競爭力

5. MacBERTWithGRU ✅文字（MacBERT）+ ✅數值
文字處理： MacBERT 之後再串 GRU
融合方式： GRU 輸出最後一步拼接數值特徵
意圖： 想看 BERT+RNN 的表現 vs. 傳統 BERT

6. MacBERTMLPFusion ✅文字 + ✅數值
處理方式： 文字與數值直接拼接後進入 MLP
不同於 FusionMacBERT：
沒有額外處理數值特徵（如沒有經過 nn.Linear)
更單純的融合設計（屬於 Early Fusion）

7. TextCNNMacBERT ✅文字 + ✅數值
模型組合：
使用 BERT 編碼後丟進 CNN filter (TextCNN)
再與數值特徵融合
用途： 測試 BERT 結合 CNN 特徵提取是否提升效果
有趣點： 有些短文模型（如微博、Threads）對 CNN 特徵抓取敏感

8. RoBERTa ✅文字 + ✅數值
BERT 替代品： 改用 RoBERTa（中文版本）
融合方式： 同 FusionMacBERT
實驗目的： 測試不同語言模型對結果的影響（語言模型 ablation）


9. BERTwwmExt ✅文字 + ✅數值
BERT： 使用 Chinese BERT whole-word-masking 擴展版
比較目的： 同上，用於測試不同語言模型特性的影響

10. ERNIE ✅文字 + ✅數值
BERT： 改用百度的 ERNIE（引入知識增強）
適用場景： 當文本與常識有關（如話題、用語）
目的： 評估知識型語言模型在社群文本分類的效果

11. ConvBERT ✅文字 + ✅數值
模型特色： 使用 Convolution + Self-Attention 混合架構的 BERT
實驗意義： 試驗非傳統 Self-Attention 模型是否有優勢


| 模型名稱              | 說明               | 是否融合 | 文本處理法         | 特殊處理       |
| ----------------- | ---------------- | ---- | ------------- | ---------- |
| FusionMacBERT     | BERT + 數值特徵      | ✅    | MacBERT       | 自製融合層      |
| PureMacBERT       | 純文本模型            | ❌    | MacBERT       | baseline   |
| NumericOnly       | 純統計數值            | ❌    | 無             | MLP only   |
| BiLSTMWithNumeric | LSTM + 數值        | ✅    | nn.Embedding  | 不使用 BERT   |
| MacBERTWithGRU    | BERT + GRU + 數值  | ✅    | MacBERT + GRU | 時序特徵強化     |
| MacBERTMLPFusion  | BERT + 數值        | ✅    | MacBERT       | 拼接後進 MLP   |
| TextCNNMacBERT    | BERT + CNN + 數值  | ✅    | MacBERT + CNN | 模仿 TextCNN |
| RoBERTa           | 換 BERT backbone  | ✅    | RoBERTa       | 模型比較       |
| BERTwwmExt        | 換 BERT backbone  | ✅    | BERT-wwm      | 模型比較       |
| ERNIE             | 引入知識的 BERT       | ✅    | ERNIE         | 模型比較       |
| ConvBERT          | 混合卷積 + 注意力的 BERT | ✅    | ConvBERT      | 模型比較       |




In [ ]:

'''
# 資料分割：資料集切分與取樣
dataset = CustomDataset(df)
train_size = int(0.8 * len(dataset))
train_dataset, test_dataset = random_split(dataset, [train_size, len(dataset)-train_size])
train_labels = [train_dataset[i]['labels'].item() for i in range(len(train_dataset))]
class_counts = pd.Series(train_labels).value_counts().to_dict()
weights = [1.0 / class_counts[label] for label in train_labels]
sampler = WeightedRandomSampler(weights, len(weights), replacement=True)
train_loader = DataLoader(train_dataset, batch_size=8, sampler=sampler)
test_loader = DataLoader(test_dataset, batch_size=8)

# 執行多模型訓練
model_variants = {
    "FusionMacBERT": FusionMacBERTModel("hfl/chinese-macbert-base", len(num_cols), 3),
    "PureMacBERT": PureMacBERTModel("hfl/chinese-macbert-base", 3),
    "NumericOnly": NumericOnlyModel(len(num_cols), 3),
    "BiLSTMWithNumeric": BiLSTMWithNumeric(tokenizer.vocab_size, 128, 128, len(num_cols), 3),
    "MacBERTWithGRU": MacBERTWithGRU("hfl/chinese-macbert-base", len(num_cols), 3),
    "MacBERTMLPFusion": MacBERTMLPFusion("hfl/chinese-macbert-base", len(num_cols), 3),
    "TextCNNMacBERT": TextCNNMacBERT("hfl/chinese-macbert-base", len(num_cols), 3),
    "RoBERTa": FusionMacBERTModel("hfl/chinese-roberta-wwm-ext", len(num_cols), 3),
    "BERTwwmExt": FusionMacBERTModel("hfl/chinese-bert-wwm-ext", len(num_cols), 3),
    "ERNIE": FusionMacBERTModel("nghuyong/ernie-3.0-base-zh", len(num_cols), 3),
    "ConvBERT": FusionMacBERTModel("YituTech/conv-bert-base", len(num_cols), 3)
}

# 逐個模型訓練與輸出結果
for name, model in model_variants.items():
    tokenizer_name = model_tokenizer_map.get(name, default_tokenizer_name)
    if tokenizer_name:
        tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
        encodings = tokenizer(df['content'].tolist(), truncation=True, padding='max_length', max_length=128)
        df['input_ids'] = encodings['input_ids']
        df['attention_mask'] = encodings['attention_mask']
    train_and_eval(model, name)
 '''

# 迴歸預測

In [ ]:
# 原本這樣分類（要拿掉）
# df['view_class'] = ...
# df['label'] = ...

# 直接用原始 view_count 作為 regression target
df = df.dropna(subset=["content", "view_count"])
df['target'] = df['view_count'].apply(parse_count)  # 如果 view_count 不是數字要先轉換


In [ ]:
def train_and_eval_regression(model, name):
    model = model.to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
    loss_fn = nn.MSELoss()

    # 訓練
    for epoch in range(5):
        model.train()
        for batch in train_loader:
            optimizer.zero_grad()
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            numerics = batch['numerics'].to(device)
            targets = batch['labels'].float().to(device)  # 重要：labels 必須是 float
            preds = model(input_ids, attention_mask, numerics)
            loss = loss_fn(preds, targets)
            loss.backward()
            optimizer.step()

    # 評估
    model.eval()
    all_preds, all_targets = [], []
    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            numerics = batch['numerics'].to(device)
            targets = batch['labels'].float().to(device)
            preds = model(input_ids, attention_mask, numerics)
            all_preds.extend(preds.cpu().numpy())
            all_targets.extend(targets.cpu().numpy())

    from sklearn.metrics import mean_squared_error, mean_absolute_error
    mse = mean_squared_error(all_targets, all_preds)
    mae = mean_absolute_error(all_targets, all_preds)
    r2 = r2_score(all_targets, all_preds)
    print(f"\n{name}  MSE: {mse:.2f} | MAE: {mae:.2f}| R2: {r2:.2f}")

    from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

    return all_targets, all_preds





In [ ]:
class FusionMacBERTRegressor(nn.Module):
    def __init__(self, model_name, num_numeric_features, output_dim=1):
        super().__init__()
        self.bert = AutoModel.from_pretrained(model_name)
        self.dropout = nn.Dropout(0.3)
        self.numeric_fc = nn.Linear(num_numeric_features, 64)
        self.regressor = nn.Linear(self.bert.config.hidden_size + 64, output_dim)

    def forward(self, input_ids, attention_mask, numerics):
        cls_output = self.bert(input_ids=input_ids, attention_mask=attention_mask).last_hidden_state[:, 0, :]
        num_out = torch.relu(self.numeric_fc(numerics))
        combined = torch.cat((cls_output, num_out), dim=1)
        return self.regressor(self.dropout(combined)).squeeze(1)  # (batch,)


In [ ]:
rm -rf ~/.cache/huggingface/transformers/hfl__chinese-macbert-base


In [ ]:
# Dataset
'''
class CustomDatasetRegression(Dataset):
    def __init__(self, df):
        self.input_ids = df['input_ids'].tolist()
        self.attention_mask = df['attention_mask'].tolist()
        self.labels = df['label'].astype(float).values   # ← 為回歸任務需轉成 float
        self.numerics = df[num_cols].astype(float).values  # ← 確保為 float array

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            'input_ids': torch.tensor(self.input_ids[idx], dtype=torch.long),
            'attention_mask': torch.tensor(self.attention_mask[idx], dtype=torch.long),
            'numerics': torch.tensor(self.numerics[idx], dtype=torch.float),  # ← 修正為 float
            'labels': torch.tensor(self.labels[idx], dtype=torch.float)       # ← 修正為 float
        }
'''
class CustomDatasetRegression(Dataset):
    def __init__(self, df):
        self.input_ids = df['input_ids'].tolist()
        self.attention_mask = df['attention_mask'].tolist()
        self.labels = df['target'].astype(float).values
        self.numerics = df[num_cols].astype(float).values

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            'input_ids': torch.tensor(self.input_ids[idx], dtype=torch.long),
            'attention_mask': torch.tensor(self.attention_mask[idx], dtype=torch.long),
            'numerics': torch.tensor(self.numerics[idx], dtype=torch.float),
            'labels': torch.tensor(self.labels[idx], dtype=torch.float)
        }

# Loss
class FocalLoss(nn.Module):
    def __init__(self, gamma=2.0):
        super().__init__()
        self.gamma = gamma
        self.ce = nn.CrossEntropyLoss()

    def forward(self, input, target):
        logp = self.ce(input, target)
        p = torch.exp(-logp)
        loss = (1 - p) ** self.gamma * logp
        return loss.mean()

In [ ]:

dataset = CustomDatasetRegression(df)
train_size = int(0.8 * len(dataset))
train_dataset, test_dataset = random_split(dataset, [train_size, len(dataset) - train_size])
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8)


class FusionMacBERTRegressor(nn.Module):
    def __init__(self, model_name, num_numeric_features, output_dim=1):
        super().__init__()
        self.bert = AutoModel.from_pretrained(model_name)
        self.dropout = nn.Dropout(0.3)
        self.numeric_fc = nn.Linear(num_numeric_features, 64)
        self.regressor = nn.Linear(self.bert.config.hidden_size + 64, output_dim)

    def forward(self, input_ids, attention_mask, numerics):
        cls_output = self.bert(input_ids=input_ids, attention_mask=attention_mask).last_hidden_state[:, 0, :]
        num_out = torch.relu(self.numeric_fc(numerics))
        combined = torch.cat((cls_output, num_out), dim=1)
        return self.regressor(self.dropout(combined)).squeeze(1)


def train_and_eval_regression(model, name):
    model = model.to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
    loss_fn = nn.MSELoss()

    for epoch in range(5):
        model.train()
        for batch in train_loader:
            optimizer.zero_grad()
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            numerics = batch['numerics'].to(device)
            targets = batch['labels'].float().to(device)

            preds = model(input_ids, attention_mask, numerics)
            loss = loss_fn(preds, targets)
            loss.backward()
            optimizer.step()

    model.eval()
    all_preds, all_targets = [], []
    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            numerics = batch['numerics'].to(device)
            targets = batch['labels'].float().to(device)

            preds = model(input_ids, attention_mask, numerics)
            all_preds.extend(preds.cpu().numpy())
            all_targets.extend(targets.cpu().numpy())

    from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
    mse = mean_squared_error(all_targets, all_preds)
    mae = mean_absolute_error(all_targets, all_preds)
    r2 = r2_score(all_targets, all_preds)

    print(f"\n{name}  MSE: {mse:.2f} | MAE: {mae:.2f} | R²: {r2:.2f}")
    return all_targets, all_preds


model = FusionMacBERTRegressor("hfl/chinese-macbert-base", len(num_cols))
all_targets, all_preds = train_and_eval_regression(model, "FusionMacBERTRegressor")

In [ ]:

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

#person_view
df["person_view_encoded"] = LabelEncoder().fit_transform(df["person_view"])

#特徵變數與目標變數
features = ["like_count", "reply_count", "emoji_count", "sentiment_score", "person_view_encoded"]
X = df[features]
y = df["viral"]

#分割資料
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

#評估模型
print(classification_report(y_test, y_pred))
